## Backwards Selection

Problem:
We have a lot of variables and we need to decide which to use in our regression model. 

Solution:
Begin with a model that contains all variables. Then, one at a time, remove the least significant variable and record AIC, BIC, and R-Squared each time we do. R-Squared measures how much of the data's variation is explained by the model. AIC and BIC measure the opposite: they measure  unexplained variance in the regression model. But they also penalize a model for adding too many variables. At the end, we use the model with the best AIC, BIC, or R-Squared.

In [1]:
import os
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm

raw0 = pd.read_csv('/Users/lukastaylor/Dropbox/Misc/Old Classes/MSEA Semester 1/ECON 5813/Data/Boston.csv')

print(raw0.head())

# define y and X
raw0 = raw0.iloc[:,1:]
#raw0np = raw0.values
Y = raw0.iloc[:,-1] # -1 means the last one
X = raw0.iloc[:,:-1] 
ncol=X.shape[1]

   Unnamed: 0     crim    zn  indus  chas    nox     rm   age     dis  rad  \
0           1  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1   
1           2  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2   
2           3  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2   
3           4  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3   
4           5  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3   

   tax  ptratio   black  lstat  medv  
0  296     15.3  396.90   4.98  24.0  
1  242     17.8  396.90   9.14  21.6  
2  242     17.8  392.83   4.03  34.7  
3  222     18.7  394.63   2.94  33.4  
4  222     18.7  396.90   5.33  36.2  


In [9]:
detailed = input("Would you like to have detailed information? (y/n)").lower()

while True: 
    criteria = input("What would criteria would you like to use? (AIC/BIC/Rsquared)").lower()
    mode = {"aic":2,"bic":1,"rsquared":0,"r-squared":0}[criteria]
    if mode in range(0,5): break
    else: print("Error: invalid criteria")

# Code for backwards selection based on AIC, BIC, or R-Squared

pcand = list(range(ncol)) # A list to keep track of the remaining predictors (i.e., not yet removed) at each iteration
psel = list(range(ncol)) # A list to keep track of the selected predictors at each iteration (the order of the selected predictors)
tb = np.zeros(ncol) # A vector to store the AIC, BIC, or R-Squared of the selected model (combination) at each iteration
p = 0 # Iteration idex
removelist, fittedlist = [], []

while p != ncol: # Repeat below until the model excludes all the predictors
    
    tb0 = np.zeros((len(pcand),3)) # Store the Rsquared(s), AIC(s),  and BIC(s) of the models considered at each iteration
    fittedlisttemp=[]

    for i in range(0,len(pcand)):
        
        #Try removing a variable and store resulting AIC, BIC, or R-Squared 
        pseltemp=psel
        pseltemp.remove(pcand[i])
        res = sm.OLS(Y,  sm.add_constant(X.iloc[:, pseltemp])).fit()
        tb0[i,:] = [res.rsquared, res.bic, res.aic]
        fittedlisttemp.append(res)
        psel.insert(0, pcand[i])
        psel.sort()
        
        if detailed == "y": print("\nTried removing the following variable:", pcand[i], "\nincluded predictors:", pseltemp, "\nAIC:", tb0[i,2], "BIC:", tb0[i,1], "R-Squared:", tb0[i,0])
    
    # Find the regressor that results in the best criteria when added to the model
    ind = np.argmax(tb0[:,mode]) if mode == 0 else np.argmin(tb0[:,mode])
    psel.remove(pcand[ind])
    p += 1
    
    if detailed == "y": print("\nRemoving", pcand[ind], "\nWe have gone through the while statement", p, "time(s)\n--------------------------------")
    
    removelist.append(pcand[ind])
    pcand.remove(pcand[ind]) # Remove the selected regressor from pcand
    tb[p-1] =  tb0[ind,mode] # Store the AIC, BIC, or R-Squared of the selected model at this iteration
    fittedlist.append(fittedlisttemp[ind])

#Finding Best Model
best = np.argmax(tb)+1 if mode == 0 else np.argmin(tb)+1

# Printing Results
print("--------------------------\nDone.")
if detailed == "y": x = [print("\nMODEL NUMBER:", i, "\nThis Model Removed Variable:", removelist[i], "\nCriteria", criteria.upper(), "Value:", tb[i]) for i in range(len(tb))]; x
print("\nFINAL MODEL INFORMATION\n\nFinal Model Kept These Variables (in order of removal):")
for i in removelist[best:]: print("Variable Numer #"+str(i)+" - " + raw0.columns[i+1])
print("\nFinal Model Removed These Variables:")
for i in removelist[:best]: print("Variable Number #"+str(i)+" - " + raw0.columns[i+1]) 
print(""); print(str("Final Model's "+criteria.upper())+str(": ")+str(tb[best]))

Would you like to have detailed information? (y/n) n
What would criteria would you like to use? (AIC/BIC/Rsquared) bic


--------------------------
Done.

FINAL MODEL INFORMATION

Final Model Kept These Variables (in order of removal):
Variable Numer #3 - nox
Variable Numer #1 - indus
Variable Numer #9 - ptratio
Variable Numer #0 - zn
Variable Numer #8 - tax
Variable Numer #11 - lstat
Variable Numer #4 - rm
Variable Numer #7 - rad
Variable Numer #10 - black
Variable Numer #5 - age
Variable Numer #12 - medv

Final Model Removed These Variables:
Variable Number #6 - dis
Variable Number #2 - chas

Final Model's BIC: 3076.4884435565546


## Regression Of Best Model

In [10]:
print("AIC is", fittedlist[best].aic, "\nBIC is", fittedlist[best].bic, "\nR-Squared is", fittedlist[best].rsquared_adj, "\n")
print(fittedlist[best].summary())

AIC is 3029.9965401943923 
BIC is 3076.4884435565546 
R-Squared is 0.7299149280771855 

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     137.5
Date:                Sun, 23 Apr 2023   Prob (F-statistic):          6.70e-136
Time:                        01:37:19   Log-Likelihood:                -1504.0
No. Observations:                 506   AIC:                             3030.
Df Residuals:                     495   BIC:                             3076.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------